In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [41]:
#Test webu, vytvoreni tabulky a ziskani nadpisu
url = "https://wiki.uibk.ac.at/noscemus/A_Latin_Letter_containing_some_Animadversions_upon_Mr._Isaac_Newton,_his_Theory_of_Light"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", class_="wikitable")
    row_data = [item.get_text() for item in table.find_all("td")]
    columns = [item.text.strip() for item in table.find_all("th")]
    #metadata_tabulka = pd.DataFrame(data=row_data, index=columns)
else:
    print("Chyba, response code je:", response.status_code)

In [ ]:
row_data

In [3]:
metadata_tabulka

,Author,Full title,In,Year,Place,Publisher/Printer,Era,Form/Genre,Discipline/Content,Digital copies,Description,References,Cited in,How to cite this entry,Internal notes


In [9]:
# zatim nefunguje - work in progress !!!
url = "https://wiki.uibk.ac.at/noscemus/Category:Works"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

names = soup.find("div", class="mw-category-generated")


print(names)

'''
table = soup.find("table", class_="wikitable")
rows = table.find_all("tr")
columns = [item.text.strip() for item in table.find_all("th")]
metadata_tabulka = pd.DataFrame(columns=columns)
'''

SyntaxError: invalid syntax (3601109673.py, line 6)